In [79]:
import os
import ee
import geemap
import geemap.chart as chart
from geemap import cartoee
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [80]:
# Get basemap layer and set it to Addis Ababa
map = geemap.Map(center=(8.9801, 38.7805), zoom=11)
map

Map(center=[8.9801, 38.7805], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [81]:
import geemap

Map = geemap.Map()

In [82]:
# Get Addis boundary shapefile from https://data.humdata.org/dataset/cod-ab-eth
addis_shpfile = "C:/Users/c1032379/OneDrive - Newcastle University/10_Research_Project/Project_MRes/data/input_data/admin_boundaries/addis_bndry_py_hdx_2021.shp"
addis_bndry = geemap.shp_to_ee(addis_shpfile)

start = ee.Date("2020-01-01")
date_today = ee.Date(datetime.datetime.today().strftime('%Y-%m-%d'))

# Set area of interest covering Addis Ababa
aoi = ee.Geometry.Point([38.7805, 8.9801])

def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)

def LSmask(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)    

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True).updateMask(qaMask)

images_Sent = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate(start, date_today) \
    .filter('CLOUDY_PIXEL_PERCENTAGE < 2') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .map(se2mask).median().clip(addis_bndry)


images_LS = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(start, date_today) \
    .filter('CLOUD_COVER < 2') \
    .sort('CLOUD_COVER') \
    .map(LSmask).median().clip(addis_bndry)

In [83]:
ls_vis = {"min": 0, "max": 0.4,  "gamma": 1.2, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}
sent_vis = {"min":0.0, "max":0.3, 'bands': ['B4', 'B3', 'B2']}

map.addLayer(images_Sent, sent_vis, 'Sentinel')
map.addLayer(images_LS, ls_vis, 'Landsat')